In [1]:
from googletrans import Translator
import codecs
import copy 

In [2]:
train_file = 'D:/PythoProjects/Datasets/conll003/conll003-englishversion/train.txt'
dev_file='D:/PythoProjects/Datasets/conll003/conll003-englishversion/valid.txt'
test_file='D:/PythoProjects/Datasets/conll003/conll003-englishversion/test_2.txt'


In [24]:
def read_corpus(lines):
    translator = Translator()
    
    lineText = ""
    lineTextTemp = ""
    tagText = ""
    tagTextTemp = ""
    previousTag = "O"
    
    translatedList = []
    translatedList.append('-DOCSTART- -X- -X- O')
    translatedList.append('')
    
    lineTags = []
    for line in lines:
        if not (line.isspace() or (len(line) > 10 and line[0:10] == '-DOCSTART-')):
            line = line.rstrip('\n').split()
            if (len(lineTextTemp) > 0) and line[0] != "." and line[0] != ",":
                lineTextTemp += " "
            lineTextTemp += line[0]
            
            if line[-1].startswith('B-') or line[-1].startswith('O'):
                if len(tagTextTemp) > 0:
                    tagText = copy.deepcopy(tagTextTemp)
                    translation =translator.translate(tagText,dest="tr")
                    #print(translation.origin, ' -> ', translation.text) 
                    newTags = translation.text.rstrip('\n').split()
                    counter = 1
                    for newTag in newTags:
                        tagBegin = 'B-'
                        if counter > 1:
                            tagBegin = 'I-'
                        lineTags.append(newTag + ' ' + tagBegin + previousTag)
                        print(newTag + ' ' + tagBegin + previousTag)
                        counter = counter + 1
                        
                    if line[-1].startswith('B-'):
                        tagTextTemp = line[0]
                        
                        if 'PER' in line[-1]:
                            previousTag = 'PER'
                        elif 'LOC' in line[-1]:
                            previousTag = 'LOC'
                        elif 'ORG' in line[-1]:
                            previousTag = 'ORG'
                    else:
                        tagTextTemp = ''
                        previousTag = 'O'
                    
                
            elif line[-1].startswith('I-'):
                if (len(tagTextTemp) > 0) and line[0] != "." and line[0] != ",":
                    tagTextTemp += " "
                tagTextTemp += line[0]
            
            
        elif (line.isspace() and len(lineTextTemp) > 0):
            print('space ----------------------------------')
            lineText = copy.deepcopy(lineTextTemp)
            translation =translator.translate(lineText,dest="tr")
            #print(translation.origin, ' -> ', translation.text) 
            targetLanguageText = translation.text
            targetLanguageWords = targetLanguageText.rstrip('\n').split()
            for targetLanguageWord in targetLanguageWords:
                currentTag = 'O'
                #for lineTag in lineTags:
                    #print(lineTag)
                matching = [s for s in lineTags if any(targetLanguageWord in s for xs in lineTags)]
                if len(matching) > 0:
                    currentWord = matching.rstrip('\n').split()
                    currentTag = currentWord[-1]
                translatedList.append(targetLanguageWord + ' ' + currentTag)
                #print(targetLanguageWord + ' ' + currentTag)
            
            translatedList.append('')
            lineTextTemp = ""
        
    

    return translatedList

In [25]:
# load corpus
print('loading corpus')
with codecs.open(train_file, 'r', 'utf-8') as f:
    lines = f.readlines()
with codecs.open(dev_file, 'r', 'utf-8') as f:
    dev_lines = f.readlines()
with codecs.open(test_file, 'r', 'utf-8') as f:
    test_lines = f.readlines()

test_features, test_labels = read_corpus(test_lines)

loading corpus
space ----------------------------------
space ----------------------------------
Kız B-O
göz B-LOC
Arap B-O
Emirlikleri I-O
space ----------------------------------


AttributeError: 'list' object has no attribute 'rstrip'

In [ ]:
print(test_features)
print(test_labels)

In [ ]:
translations =translator.translate(['soccer - japan get lucky win, china in suprise defeat.','japan began the defence of their asian cup title with a lucky 2-1 win against syria in a group c championship match on friday.'],dest="tr")
for translation in translations:
    print(translation.origin, ' -> ', translation.text)